<a href="https://colab.research.google.com/github/Pranav335/Explainable-Recommendations/blob/main/context_aware_cv_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flash_attn==2.5.8
!pip install torch==2.3.1
!pip install accelerate==0.31.0
!pip install transformers==4.41.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/s eta 0:00:00
 

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
device

'cuda'

In [25]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct",
                                          padding_size = 'left',
    add_eos_token = True, # Automatically add eos at each row so that it generates data in controlled manner.
    add_bos_token = True,
    use_fast = False)

In [5]:
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
job_description = '''
Position Title: Java Developer
Location: [Company Location]
Job Type: Full-time

Job Summary:
We are seeking an experienced Java Developer to join our dynamic development team.
The ideal candidate will be responsible for designing, implementing, and maintaining Java applications
that are high-volume, low-latency, and mission-critical. The Java Developer will contribute to the
entire software development lifecycle, from concept and design to testing.

Key Responsibilities:
- Develop, test, and maintain high-performance Java applications.
- Design and implement efficient, scalable, and secure code.
- Collaborate with cross-functional teams to define, design, and ship new features.
- Participate in code reviews and provide constructive feedback to improve quality.
- Write well-designed, efficient, and testable code following best practices.
- Troubleshoot, debug, and resolve production issues.
- Ensure continuous integration and automated testing in the development process.
- Contribute to architectural discussions and decisions.

Qualifications:
- Bachelor’s degree in Computer Science, Engineering, or a related field.
- 3+ years of experience in Java development.
- Strong understanding of object-oriented programming (OOP) principles and design patterns.
- Proficiency with Java frameworks like Spring and Hibernate.
- Experience with RESTful APIs, Microservices, and relational databases (SQL).
- Knowledge of front-end technologies like HTML, CSS, and JavaScript is a plus.
- Familiarity with version control systems like Git.
- Strong problem-solving and analytical skills.
- Excellent communication and teamwork abilities.
'''

In [12]:
resume = '''
Name: John Doe
Address: [Your Address]
Phone: [Your Phone Number]
Email: [Your Email Address]
LinkedIn: [Your LinkedIn Profile]

Professional Summary:
Experienced Java Developer with over 4 years of experience designing and developing enterprise-level
Java applications. Proficient in object-oriented programming, Java frameworks, and RESTful services.
Adept at collaborating with cross-functional teams to deliver high-quality software solutions
with a focus on scalability and performance.

Technical Skills:
- Programming Languages: Java, Python, SQL
- Java Frameworks: Spring, Hibernate, JPA
- Web Technologies: HTML, CSS, JavaScript
- Databases: MySQL, PostgreSQL
- Tools & Platforms: Maven, Gradle, Docker, Jenkins, Git
- Cloud Services: AWS, Google Cloud Platform
- Other: Microservices Architecture, RESTful APIs, Agile Development

Professional Experience:

Java Developer
TechCorp Solutions, City, State
April 2021 – Present
- Developed and maintained high-volume Java applications used by thousands of customers.
- Worked on both front-end and back-end development using Java, Spring Boot, and MySQL.
- Designed and implemented RESTful APIs for various business services.
- Collaborated with the DevOps team to implement continuous integration and deployment pipelines.
- Optimized SQL queries and Java code to improve application performance by 20%.

Junior Java Developer
SoftTech Innovations, City, State
June 2019 – March 2021
- Assisted in the design and development of Java-based applications, contributing to both coding and testing.
- Integrated Java applications with external REST APIs to fetch and process data in real-time.
- Participated in code reviews and optimized existing code for performance and scalability.
- Implemented security features using Spring Security to protect sensitive data.

Education:
Bachelor of Science in Computer Science
University of Technology, City, State
Graduated: May 2019

Certifications:
- Oracle Certified Professional: Java SE Programmer (2020)
- AWS Certified Solutions Architect – Associate (2022)

References:
Available upon request.'''


In [23]:
content = f'''This is a resume : {resume} \n
This is a job description: {job_description} \n
Summarize the resume in context of job description'''

In [27]:
message = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": content},
]

In [31]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    'max_length':4000,
    "return_full_text": True,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(message, **generation_args)
print(output[0]['generated_text'])

[{'role': 'system', 'content': 'You are a helpful AI assistant.'}, {'role': 'user', 'content': 'Can you provide ways to eat combinations of bananas and dragonfruits?'}, {'role': 'assistant', 'content': 'Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey.'}, {'role': 'user', 'content': 'This is a resume : \nName: John Doe\nAddress: [Your Address]\nPhone: [Your Phone Number]\nEmail: [Your Email Address]\nLinkedIn: [Your LinkedIn Profile]\n\nProfessional Summary:\nExperienced Java Developer with over 4 years of experience designing and developing enterprise-level\nJava applications. Proficient in object-oriented programming, Java frameworks, and RESTful services.\nAdept at collaborating with cross-functional teams to deliver high-quality software solutions\nwith a 

In [32]:
output

[{'generated_text': [{'role': 'system',
    'content': 'You are a helpful AI assistant.'},
   {'role': 'user',
    'content': 'Can you provide ways to eat combinations of bananas and dragonfruits?'},
   {'role': 'assistant',
    'content': 'Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey.'},
   {'role': 'user',
    'content': 'This is a resume : \nName: John Doe\nAddress: [Your Address]\nPhone: [Your Phone Number]\nEmail: [Your Email Address]\nLinkedIn: [Your LinkedIn Profile]\n\nProfessional Summary:\nExperienced Java Developer with over 4 years of experience designing and developing enterprise-level\nJava applications. Proficient in object-oriented programming, Java frameworks, and RESTful services.\nAdept at collaborating with cross-functional teams to del

In [45]:
content = f'''This is a resume : {resume} \n
This is a job description: {job_description} \n
Summarize the most similar aspect of resume to the job description'''

In [46]:
message = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": content},
]

In [47]:
output = pipe(message, **generation_args)

In [48]:
output

[{'generated_text': [{'role': 'system',
    'content': 'You are a helpful AI assistant.'},
   {'role': 'user',
    'content': 'Can you provide ways to eat combinations of bananas and dragonfruits?'},
   {'role': 'assistant',
    'content': 'Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey.'},
   {'role': 'user',
    'content': 'This is a resume : \nName: John Doe\nAddress: [Your Address]\nPhone: [Your Phone Number]\nEmail: [Your Email Address]\nLinkedIn: [Your LinkedIn Profile]\n\nProfessional Summary:\nExperienced Java Developer with over 4 years of experience designing and developing enterprise-level\nJava applications. Proficient in object-oriented programming, Java frameworks, and RESTful services.\nAdept at collaborating with cross-functional teams to del

In [53]:
content = f'''This is a resume : {resume} \n
This is a job description: {job_description} \n
Summarize the most similar aspect of resume to the job description in one line without mentioning word resume'''

In [54]:
message = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": content},
]

In [55]:
output = pipe(message, **generation_args)

In [56]:
output

[{'generated_text': [{'role': 'system',
    'content': 'You are a helpful AI assistant.'},
   {'role': 'user',
    'content': 'Can you provide ways to eat combinations of bananas and dragonfruits?'},
   {'role': 'assistant',
    'content': 'Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey.'},
   {'role': 'user',
    'content': 'This is a resume : \nName: John Doe\nAddress: [Your Address]\nPhone: [Your Phone Number]\nEmail: [Your Email Address]\nLinkedIn: [Your LinkedIn Profile]\n\nProfessional Summary:\nExperienced Java Developer with over 4 years of experience designing and developing enterprise-level\nJava applications. Proficient in object-oriented programming, Java frameworks, and RESTful services.\nAdept at collaborating with cross-functional teams to del

In [61]:
content = f'''This is a resume : {resume} \n
This is a job description: {job_description} \n
Summarize the most similar aspect of resume to the job description without mentioning word resume and job description'''

In [62]:
message = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": content},
]

In [65]:
%%time
output = pipe(message, **generation_args)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


CPU times: user 11.3 s, sys: 750 ms, total: 12.1 s
Wall time: 12.1 s


In [64]:
output

[{'generated_text': [{'role': 'system',
    'content': 'You are a helpful AI assistant.'},
   {'role': 'user',
    'content': 'Can you provide ways to eat combinations of bananas and dragonfruits?'},
   {'role': 'assistant',
    'content': 'Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey.'},
   {'role': 'user',
    'content': 'This is a resume : \nName: John Doe\nAddress: [Your Address]\nPhone: [Your Phone Number]\nEmail: [Your Email Address]\nLinkedIn: [Your LinkedIn Profile]\n\nProfessional Summary:\nExperienced Java Developer with over 4 years of experience designing and developing enterprise-level\nJava applications. Proficient in object-oriented programming, Java frameworks, and RESTful services.\nAdept at collaborating with cross-functional teams to del